**HOURGLASS-SHAPED CNN - number of filters will be 15 12 9 6 3 3 6 9 12 15 as an hourglass shaped)**


# INITIALIZATION

In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import tensorboard
import matplotlib.pyplot as plt

import numpy as np

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
train_images = train_images / 255.0 #normalizing
print(train_images.shape)
print(train_labels.shape)

(60000, 28, 28)
(60000,)


In [3]:
fig, ax = plt.subplots(2,5)
ax = ax.flatten()
for i in range(10):
    im_idx = np.argwhere(train_labels == i)[0]
    plottable_image = np.reshape(train_images[im_idx], (28, 28))
    ax[i].imshow(plottable_image, cmap='gray')

In [4]:
x_train = train_images.reshape(-1, 28, 28, 1) #add an additional dimension to represent the single-channel
x_test = test_images.reshape(-1, 28, 28, 1)
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


For testing different parameters in the model, I will run it through the parameters as below:
- Learning rate 0.0005, 0.001, 0.01, 0.05, 0.1
- Batch size from 2^6 to 2^10 (64, 128, 256, 512, 1024)
- Optimization: SGD, adam, RMSProp

I set a multiple value of 3 so I can loop through and set the number of filters in each layer of the CNN.

# LEARNING RATE OPTIMIZATION
Set batch size = 256

In [5]:
learning_rate = [0.0005, 0.001, 0.01, 0.05, 0.1]
a = 3
for b in range(8,9):
    for j in range(0,5):
        print('With 10 epochs, learning rate: ' + str(learning_rate[j])+ ', batch size:' + str(2**b))
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate[j])
        cnn_model = tf.keras.models.Sequential()
        for i in range(1,6):
                if i%2==0:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*(6-i), kernel_size = 3, strides=(1, 1), padding='same', activation='relu'))
                    if i<9:
                        cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
                else:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*(6-i), kernel_size = 3, padding='same', activation='relu'))
        for i in range(1,6):
                if i%2==1:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*i, kernel_size = 3, strides=(1, 1), padding='same', activation='relu'))
                    if i<4:
                        cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
                else:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*i, kernel_size = 3, padding='same', activation='relu'))
        cnn_model.add(tf.keras.layers.Flatten())
        cnn_model.add(tf.keras.layers.Dense(256))
        cnn_model.add(tf.keras.layers.Activation('relu'))
        cnn_model.add(tf.keras.layers.Dense(10))
        cnn_model.add(tf.keras.layers.Activation('softmax'))

        cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, 
                          metrics=['accuracy'])
        cnn_model.build(input_shape=(1,28,28,1))
        cnn_model.summary()

        # Define the Keras TensorBoard callback.
        logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

        model1 = cnn_model
        # Train the model.
        model1.fit(
            x_train,
            train_labels, 
            batch_size=2**b,
            epochs=10, 
            callbacks=[tensorboard_callback])

        # Evaluate
        score = model1.evaluate(x_test, test_labels)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

With 10 epochs, learning rate: 0.0005, batch size:256
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (1, 28, 28, 15)           150       
                                                                 
 conv2d_1 (Conv2D)           (1, 28, 28, 12)           1632      
                                                                 
 max_pooling2d (MaxPooling2D  (1, 14, 14, 12)          0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (1, 14, 14, 9)            981       
                                                                 
 conv2d_3 (Conv2D)           (1, 14, 14, 6)            492       
                                                                 
 max_pooling2d_1 (MaxPooling  (1, 7, 7, 6)             0         
 2

235/235 [==============================] - 2s 9ms/step - loss: 0.2749 - accuracy: 0.9157
Epoch 7/10
235/235 [==============================] - 2s 9ms/step - loss: 0.2366 - accuracy: 0.9255
Epoch 8/10
235/235 [==============================] - 2s 9ms/step - loss: 0.2055 - accuracy: 0.9359
Epoch 9/10
235/235 [==============================] - 2s 9ms/step - loss: 0.1839 - accuracy: 0.9425
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 26.9048 - accuracy: 0.9435
Test loss: 26.90484046936035
Test accuracy: 0.9434999823570251
With 10 epochs, learning rate: 0.01, batch size:256
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (1, 28, 28, 15)           150       
                                                                 
 conv2d_21 (Conv2D)          (1, 28, 28, 12)           1632      
                                          

Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
235/235 [==============================] - 3s 8ms/step - loss: 2.3042 - accuracy: 0.1064
Epoch 2/10
235/235 [==============================] - 2s 9ms/step - loss: 2.3033 - accuracy: 0.1078
Epoch 3/10
235/235 [==============================] - 2s 9ms/step - loss: 2.3034 - accuracy: 0.1090
Epoch 4/10
235/235 [==============================] - 2s 9ms/step - loss: 2.3033 - accuracy: 0.1076
Epoch 5/10
235/235 [==============================] - 2s 9ms/step - loss: 2.3036 - accuracy: 0.1077
Epoch 6/10
235/235 [==============================] - 2s 8ms/step - loss: 2.3038 - accuracy: 0.1057
Epoch 7/10
235/235 [==============================] - 2s 8ms/step - loss: 2.3032 - accuracy: 0.1090
Epoch 8/10
235/235 [==============================] - 2s 9ms/step - loss: 2.3031 - accuracy: 0.1076
Epoch 9/10
235/235 [==============================] - 2s 8ms/step - loss: 2.3036 - accuracy: 0.1073
Epoch 10/1

# BATCH SIZE
Optimum learning rate 0.001 will be used

In [6]:
learning_rate = [0.001]
a = 3
for b in range(6,11):
    for j in range(0,1):
        print('With 10 epochs, learning rate: ' + str(learning_rate[j])+ ', batch size:' + str(2**b))
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate[j])
        cnn_model = tf.keras.models.Sequential()
        for i in range(1,6):
                if i%2==0:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*(6-i), kernel_size = 3, strides=(1, 1), padding='same', activation='relu'))
                    if i<9:
                        cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
                else:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*(6-i), kernel_size = 3, padding='same', activation='relu'))
        for i in range(1,6):
                if i%2==1:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*i, kernel_size = 3, strides=(1, 1), padding='same', activation='relu'))
                    if i<4:
                        cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
                else:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*i, kernel_size = 3, padding='same', activation='relu'))
        cnn_model.add(tf.keras.layers.Flatten())
        cnn_model.add(tf.keras.layers.Dense(256))
        cnn_model.add(tf.keras.layers.Activation('relu'))
        cnn_model.add(tf.keras.layers.Dense(10))
        cnn_model.add(tf.keras.layers.Activation('softmax'))

        cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, 
                          metrics=['accuracy'])
        cnn_model.build(input_shape=(1,28,28,1))
        cnn_model.summary()

        # Define the Keras TensorBoard callback.
        logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

        model2 = cnn_model
        # Train the model.
        model2.fit(
            x_train,
            train_labels, 
            batch_size=2**b,
            epochs=10, 
            callbacks=[tensorboard_callback])

        # Evaluate
        score = model2.evaluate(x_test, test_labels)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

With 10 epochs, learning rate: 0.001, batch size:64
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_50 (Conv2D)          (1, 28, 28, 15)           150       
                                                                 
 conv2d_51 (Conv2D)          (1, 28, 28, 12)           1632      
                                                                 
 max_pooling2d_20 (MaxPoolin  (1, 14, 14, 12)          0         
 g2D)                                                            
                                                                 
 conv2d_52 (Conv2D)          (1, 14, 14, 9)            981       
                                                                 
 conv2d_53 (Conv2D)          (1, 14, 14, 6)            492       
                                                                 
 max_pooling2d_21 (MaxPoolin  (1, 7, 7, 6)             0         
 g

469/469 [==============================] - 3s 6ms/step - loss: 0.1646 - accuracy: 0.9491
Epoch 7/10
469/469 [==============================] - 3s 6ms/step - loss: 0.1489 - accuracy: 0.9539
Epoch 8/10
469/469 [==============================] - 3s 6ms/step - loss: 0.1346 - accuracy: 0.9583
Epoch 9/10
469/469 [==============================] - 3s 6ms/step - loss: 0.1246 - accuracy: 0.9611
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 47.1221 - accuracy: 0.8917
Test loss: 47.12208557128906
Test accuracy: 0.891700029373169
With 10 epochs, learning rate: 0.001, batch size:256
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_70 (Conv2D)          (1, 28, 28, 15)           150       
                                                                 
 conv2d_71 (Conv2D)          (1, 28, 28, 12)           1632      
                                          

Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
118/118 [==============================] - 3s 14ms/step - loss: 1.9650 - accuracy: 0.2343
Epoch 2/10
118/118 [==============================] - 2s 14ms/step - loss: 0.8104 - accuracy: 0.7211
Epoch 3/10
118/118 [==============================] - 2s 14ms/step - loss: 0.5018 - accuracy: 0.8381
Epoch 4/10
118/118 [==============================] - 2s 14ms/step - loss: 0.3900 - accuracy: 0.8778
Epoch 5/10
118/118 [==============================] - 2s 13ms/step - loss: 0.3191 - accuracy: 0.9005
Epoch 6/10
118/118 [==============================] - 2s 14ms/step - loss: 0.2810 - accuracy: 0.9124
Epoch 7/10
118/118 [==============================] - 2s 14ms/step - loss: 0.2433 - accuracy: 0.9254
Epoch 8/10
118/118 [==============================] - 2s 14ms/step - loss: 0.2181 - accuracy: 0.9327
Epoch 9/10
118/118 [==============================] - 2s 14ms/step - loss: 0.2046 - accuracy: 0.9372
E

# OPTIMIZER: SGD, RMSprop

Change opt = tf.keras.optimizers.Adam(learning_rate=learning_rate[j]) to:

opt = tf.keras.optimizers.SGD(learning_rate=learning_rate[j]) and

opt = tf.keras.optimizers.RMSProp(learning_rate=learning_rate[j])

In [9]:
learning_rate = [0.001]
a = 3
for b in range(6,7):
    for j in range(0,1):
        print('With 10 epochs, learning rate: ' + str(learning_rate[j])+ ', batch size:' + str(2**b))
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate[j])
        cnn_model = tf.keras.models.Sequential()
        for i in range(1,6):
                if i%2==0:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*(6-i), kernel_size = 3, strides=(1, 1), padding='same', activation='relu'))
                    if i<9:
                        cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
                else:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*(6-i), kernel_size = 3, padding='same', activation='relu'))
        for i in range(1,6):
                if i%2==1:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*i, kernel_size = 3, strides=(1, 1), padding='same', activation='relu'))
                    if i<4:
                        cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
                else:
                    cnn_model.add(tf.keras.layers.Conv2D(filters = a*i, kernel_size = 3, padding='same', activation='relu'))
        cnn_model.add(tf.keras.layers.Flatten())
        cnn_model.add(tf.keras.layers.Dense(256))
        cnn_model.add(tf.keras.layers.Activation('relu'))
        cnn_model.add(tf.keras.layers.Dense(10))
        cnn_model.add(tf.keras.layers.Activation('softmax'))

        cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, 
                          metrics=['accuracy'])
        cnn_model.build(input_shape=(1,28,28,1))
        cnn_model.summary()

        # Define the Keras TensorBoard callback.
        logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

        model2 = cnn_model
        # Train the model.
        model2.fit(
            x_train,
            train_labels, 
            batch_size=2**b,
            epochs=10, 
            callbacks=[tensorboard_callback])

        # Evaluate
        score = model2.evaluate(x_test, test_labels)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

With 10 epochs, learning rate: 0.001, batch size:64
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_110 (Conv2D)         (1, 28, 28, 15)           150       
                                                                 
 conv2d_111 (Conv2D)         (1, 28, 28, 12)           1632      
                                                                 
 max_pooling2d_44 (MaxPoolin  (1, 14, 14, 12)          0         
 g2D)                                                            
                                                                 
 conv2d_112 (Conv2D)         (1, 14, 14, 9)            981       
                                                                 
 conv2d_113 (Conv2D)         (1, 14, 14, 6)            492       
                                                                 
 max_pooling2d_45 (MaxPoolin  (1, 7, 7, 6)             0         
 